<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>


In [2]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()


*This is an initial dive into [Jupyter notebooks][jn], the self-contained Python environment.  The below post is [exported directly][ex] from [a Jupyter Notebook][nb].  The tables below each code block are the API responses.*

*I am using the [BarChart.com OnDemand API][bc] to call getHistory() and receive up to 3 months of minute-level stock price data for a basket of stocks.*  

*Then I do bar analysis to find potential buy spots.  This analysis is simply counting instances of minutely bars closing on highs.*

*After, I measure trends to find potential buy spots.  This analysis compares sequential bar highs/lows to assert trend reversals.* 

*To use the code yourself, you will need to setup a python environment, jupyter notebook or otherwise.  Then, you'll need to [request a free BarChart.com OnDemand API key][bc], and plug it into the code.*

*Last, set the file path for your project directory, and in that folder create a folder 'Stock_Price_Data' to export data to, and a folder 'assets' for your list.  In 'assets' create an Excel file 'Watchlist'.  Starting in row 4 with the header, have names in the first column and tickers in the second column.*

[jn]: https://jupyter.org/
[ex]: https://github.com/jupyter/nbconvert
[nb]: https://github.com/ale627/python/blob/master/barchart.ipynb
[bc]: https://www.barchart.com/ondemand/free-market-data-api

# Fetch the data from BarChart

Here we call the barchart getHistory API.  This is an adaptation of BlackArb's Python [tutorial][tut].

#### Steps:
- import libraries
- set directories and API key
- build URL to call API
- get_minute_data() function
- call the function

**outputs: symbol, timestamp, tradingDay, open, high, low, close, volume, openInterest**

[tut]: http://www.blackarbs.com/blog/how-to-get-free-intraday-stock-data-with-python-and-barcharts-ondemand-api/9/22/2015


In [1]:
# -*- coding: utf-8 -*-

#----------IMPORT LIBRARIES----------#
import time
t0 = time.clock()

import pandas as pd
from pandas.tseries.offsets import BDay

import numpy as np
import datetime as dt

from copy import copy
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
 

    
#----------SET DIRECTORIES----------#
    
project_dir = r'C:\Users/drale/Desktop/python/' 
price_path = project_dir + r'Stock_Price_Data\\'

# header=3 to skip metadata   
spy_components = pd.read_excel(project_dir +\
                             'assets/Watchlist.xls', header=3)
syms = spy_components.Identifier.dropna()
syms = syms.drop(syms.index[-1]).sort_values()


apikey = '99f7186682f41a3bd739984105aef52a'



#----------BUILD URL----------#

def construct_barChart_url(sym, start_date, freq, api_key=apikey):
    '''Function to construct barchart api url'''
    
    url = 'http://marketdata.websol.barchart.com/getHistory.csv?' +\
            'key={}&symbol={}&type={}&startDate={}'.format(api_key, sym, freq, start_date)
    return url




#----------F(X) DEFINITION----------#

def get_minute_data():
    '''Function to Retrieve <= 3 months of minute data for SP500 components'''
    
    # YYYY MM DD HH MM SS
    start = '20171001000000'
    #end = d
    freq = 'minutes'    
    prices = {}
    symbol_count = len(syms)
    
    try:
        for i, sym in enumerate(syms, start=1):
            api_url = construct_barChart_url(sym, start, freq, api_key=apikey)
            try:
                csvfile = pd.read_csv(api_url, parse_dates=['timestamp'])
                csvfile.set_index('timestamp', inplace=True)
                prices[sym] = csvfile
            except:
                continue

            print('{}..[fetched] | {} of {} tickers |'.format(sym, i, symbol_count)) 
    except Exception as e: 
        print(e)
    finally:
        pass
    
    px = pd.Panel.from_dict(prices)
    px.major_axis = px.major_axis.tz_localize('utc').tz_convert('US/Pacific')
    return px



#----------CALL FUNCTION----------#

pxx = get_minute_data()
pxx



# timer
secs      = np.round( ( time.clock()  - t0 ), 4 )
time_secs = "{timeSecs} seconds to run".format(timeSecs = secs)
print( time_secs )

AAPL..[fetched] | 1 of 9 tickers |
AMZN..[fetched] | 2 of 9 tickers |
BIDU..[fetched] | 3 of 9 tickers |
FB..[fetched] | 4 of 9 tickers |
GOOGL..[fetched] | 5 of 9 tickers |
GS..[fetched] | 6 of 9 tickers |
IBB..[fetched] | 7 of 9 tickers |
NFLX..[fetched] | 8 of 9 tickers |
TSLA..[fetched] | 9 of 9 tickers |
44.0071 seconds to run


# Select stock
Choose a ticker to analyze, and test the output.

In [2]:
stock = 'BIDU'

ohlc = pxx[stock]
ohlc.head(1)

,symbol,tradingDay,open,high,low,close,volume
timestamp,,,,,,,
2017-10-02 06:30:00-07:00,BIDU,2017-10-02,248.62,249.285,248.221,249.21,80053


# Accumulation
Where 1 min candlestick closes on highs, flag as accumulated.

In [3]:
#create accum column: for closing on highs, assign 1
ohlc["accum"] = np.where(ohlc["high"] == ohlc["close"], 1, 0)

#list matches
accumulation_bars = ohlc[(ohlc.accum == 1)]
accumulation_bars[["high", "close", "accum"]].tail(3)

,high,close,accum
timestamp,,,
2017-11-08 12:51:00-08:00,240.85,240.85,1
2017-11-08 12:52:00-08:00,241.08,241.08,1
2017-11-08 13:00:00-08:00,241.18,241.18,1


## Two-in-a-row
*Add the previous accumulated (or not) to current row*

Measures when two 1-min candlesticks in a row close at highs.

In [4]:
#create conf column adding prev column
ohlc["conf"] = ohlc.accum + ohlc.accum.shift(1)

#where conf is 2, make a table
confirmation = ohlc[(ohlc.conf >= 2)]
confirmation[["high", "close", "conf"]].tail(3)

,high,close,conf
timestamp,,,
2017-11-08 12:31:00-08:00,240.82,240.82,2
2017-11-08 12:32:00-08:00,241.03,241.03,2
2017-11-08 12:52:00-08:00,241.08,241.08,2


## Accumulation daily signal

*sum the consecutive accumulation by day*

**Return the 3 biggest daily signals**

In [5]:
#combine minutes into days
signal = confirmation.resample('D',how='sum')

#sort based on sum(conf)
signal.nlargest(3,'conf')

,accum,conf
timestamp,,
2017-11-08 00:00:00-08:00,32,64
2017-10-24 00:00:00-07:00,25,50
2017-10-03 00:00:00-07:00,23,46


# Trend

- See if minute bar is making new highs, new lows
- Return 1 for uptrend or downtrend
- If new trend starting, flag as buyable or sellable.

In [6]:
#create trend column based on up or not
ohlc["higher"] = ohlc.high - ohlc.high.shift(1)
ohlc["lower"] = ohlc.low.shift(1) - ohlc.low 


#where >0, give 1
#uptrend means higher highs, downtrend means lower lows, both means expansion, neither means contraction

ohlc["up_tr"] = np.where(ohlc["higher"] > 0, 1, 0)
ohlc["down_tr"] = np.where(ohlc["lower"] > 0, 1, 0)

ohlc["exp"] = np.where((ohlc["higher"]>0) & (ohlc["lower"]>0),1,0)
ohlc["cont"] = np.where((ohlc["higher"]<0) & (ohlc["lower"]<0),1,0)

#where new uptrend established, buyable.. new downtrend sellable
ohlc["buy"] = np.where((ohlc["up_tr"] == 1) & (ohlc["up_tr"].shift(1) == 0), 1, 0)
ohlc["sell"] = np.where((ohlc["down_tr"] == 1) & (ohlc["down_tr"].shift(1) == 0), 1, 0)


#ohlc.tail(3)
ohlc[["high", "low", "close", "higher", "lower", "up_tr", "down_tr", "exp", "cont", "buy", "sell"]].tail(10)


,high,low,close,higher,lower,up_tr,down_tr,exp,cont,buy,sell
timestamp,,,,,,,,,,,
2017-11-08 12:51:00-08:00,240.85,240.64,240.85,-0.24,-0.03,0,0,0,1,0,0
2017-11-08 12:52:00-08:00,241.08,240.77,241.08,0.23,-0.13,1,0,0,0,1,0
2017-11-08 12:53:00-08:00,241.16,241.02,241.105,0.08,-0.25,1,0,0,0,0,0
2017-11-08 12:54:00-08:00,241.15,241.05,241.09,-0.01,-0.03,0,0,0,1,0,0
2017-11-08 12:55:00-08:00,241.13,240.98,241.06,-0.02,0.07,0,1,0,0,0,1
2017-11-08 12:56:00-08:00,241.12,240.89,241.095,-0.01,0.09,0,1,0,0,0,0
2017-11-08 12:57:00-08:00,241.2,241,241.12,0.08,-0.11,1,0,0,0,1,0
2017-11-08 12:58:00-08:00,241.25,241.065,241.195,0.05,-0.065,1,0,0,0,0,0
2017-11-08 12:59:00-08:00,241.3,241.02,241.17,0.05,0.045,1,1,1,0,0,1


## Simulation

- Loop through the minute bars, print buys and sells.  
1 share long on long signal, and 1 share short on short signal.
This is not additive.  If there's a short on, and a long signal is received, you fully close the short and open the long.

In [7]:
# PRINTING

total = 0
buys = 0
sells = 0
prev = 0
trade = 0

for index, row in ohlc.iterrows():
    #every row list # buys and sells
    #print buys," long","and",sells,"short"

    if row['buy'] > 0 and buys == 0:
        buys = 1
        sells = 0
        if prev > 0: total = total - row['close'] + prev
            
        print trade,"long from",row['close'],"... total=",total
        
        prev = row['close']
        trade = trade + 1

    if row['sell'] == 1 and sells == 0:
        sells = 1
        buys = 0
        if prev > 0: total = total + row['close'] - prev
            
        print trade,"short from",row['close'],"... total=",total

        prev = row['close']
        trade = trade + 1
        

0 short from 248.7 ... total= 0
1 long from 249.2392 ... total= -0.5392
2 short from 249.07 ... total= -0.7084
3 long from 249.0001 ... total= -0.6385
4 short from 248.9 ... total= -0.7386
5 long from 248.43 ... total= -0.2686
6 short from 248.1 ... total= -0.5986
7 long from 247.49 ... total= 0.0113999999999
8 short from 248.745 ... total= 1.2664
9 long from 248.91 ... total= 1.1014
10 short from 249.36 ... total= 1.5514
11 long from 249.295 ... total= 1.6164
12 short from 249.57 ... total= 1.8914
13 long from 249.69 ... total= 1.7714
14 short from 249.9715 ... total= 2.0529
15 long from 250.18 ... total= 1.8444
16 short from 249.98 ... total= 1.6444
17 long from 250.06 ... total= 1.5644
18 short from 250.28 ... total= 1.7844
19 long from 250.285 ... total= 1.7794
20 short from 250.05 ... total= 1.5444
21 long from 250.315 ... total= 1.2794
22 short from 249.92 ... total= 0.8844
23 long from 249.99 ... total= 0.8144
24 short from 249.92 ... total= 0.7444
25 long from 249.78 ... total=

In [8]:
# PUSHING TO LOG[] DATAFRAME

total = 0
buys = 0
sells = 0
prev = 0
trade = 0


columns = ['L/S','price_close', 'total']
log = pd.DataFrame(columns=columns)


for index, row in ohlc.iterrows():
    #every row list # buys and sells
    #print buys," long","and",sells,"short"

    if row['buy'] > 0 and buys == 0:
        buys = 1
        sells = 0
        if prev > 0: total = total - row['close'] + prev
            
        log.set_value(trade,'L/S','long')
        log.set_value(trade,'price_close',row['close'])
        log.set_value(trade,'total',total)

        prev = row['close']
        trade = trade + 1

    if row['sell'] == 1 and sells == 0:
        sells = 1
        buys = 0
        if prev > 0: total = total + row['close'] - prev
                    
        log.set_value(trade,'L/S','short')
        log.set_value(trade,'price_close',row['close'])
        log.set_value(trade,'total',total)

        prev = row['close']
        trade = trade + 1
    
log.tail(10)
#log.sort_values(by='total',ascending=0)

,L/S,price_close,total
4089,long,240.89,-22.0398
4090,short,240.89,-22.0398
4091,long,240.885,-22.0348
4092,short,240.885,-22.0348
4093,long,240.95,-22.0998
4094,short,240.86,-22.1898
4095,long,240.63,-21.9598
4096,short,241.06,-21.5298
4097,long,241.12,-21.5898
4098,short,241.17,-21.5398


In [44]:
#statistics
total

-2.416800000001359

In [22]:
# get intraday min

recentlows = ohlc.resample('60T',how='min')
recentlows[["low"]].tail(5)

,low
timestamp,
2017-11-07 09:00:00-08:00,195.60
2017-11-07 10:00:00-08:00,195.88
2017-11-07 11:00:00-08:00,195.80
2017-11-07 12:00:00-08:00,194.80
2017-11-07 13:00:00-08:00,195.56


In [ ]:
# calc support/resistance

In [ ]:
try:
    store = pd.HDFStore(price_path + 'Minute_Symbol_Data.h5')
    store['minute_prices'] = pxx
    store.close()
except Exception as e:
    print(e)
finally:
    pass


In [ ]:
from IPython.display import display, HTML

df = pd.read_hdf(price_path + 'Minute_Symbol_Data.h5')
df
# df.select('AAPL', axis=0)
# display(df)
# pd.to_html(project_dir + 'test.html')

In [ ]:
#import matplotlib.pyplot as plt   
#from mpl_finance import candlestick_ohlc

%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9)


 
apple["close"].tail(4).plot(grid = True) 



In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go

from datetime import datetime

df = apple

trace = go.Ohlc(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close)
data = [trace]
py.iplot(data, filename='simple_ohlc')